## Train CNN model

In [14]:
import numpy as np
from scipy import misc
import imageio
from cnn import training
import tensorflow as tf
import pandas as pd

In [16]:
path_all= "dim_items（new).txt"

file_all= open(path_all, encoding="utf-8")

seq_all= re.compile("\s+")
result_all= []
for line in file_all:
    lst = seq_all.split(line.strip())
    item = {
        "item": lst[0],
        "cat": lst[1],
        "terms":lst[2:]
    }
    result_all.append(item)    
file_all.close()

keys = {}
for each in result_all:
    if  each['cat'] in keys:
        keys[ each['cat'] ] +=  [each['item']]
    else:
        keys[ each['cat'] ] = [each['item']]  

In [20]:
files = []
for i in keys:
    path = 'data/train/' + str(i)
    tmp = os.listdir(path)
    files += tmp

In [22]:
X_train = []
Y_train = []
for path in files:
    arr = imageio.imread(path) 
    # X_train.append(arr)
    X_train.append(arr[::4,::4])
    Y_train.append(label)   

In [25]:
X_org = np.array(X_train)
Y_org = np.array(Y_train)

num_org = X_org.shape[0]
train_proportion = 0.75

In [33]:
sample_idxs = np.random.choice(num_org, int(num_org*train_proportion))
train_X = X_org[sample_idxs,:]
train_Y = Y_org[sample_idxs]
print(train_X.shape)
print(train_Y.shape)

(374987, 32, 32, 3)
(374987,)


In [37]:
val_X = X_org[-sample_idxs,:]
val_Y = Y_org[-sample_idxs]

In [44]:
tf.reset_default_graph()
my_training(X_train, y_train, X_val, y_val, 
         conv_featmap=[32,64,128],
         fc_units=[256],
         conv_kernel_size=[5,5,5],
         dropout_rate=[0.5, 0.75, 1.0],
         pooling_size=[2,2,2],
         l2_norm=0.01,
         seed=235,
         learning_rate=1e-2,
         epoch=150,
         batch_size=15600,
         verbose=False,
         pre_trained_model=None)

Building my LeNet. Parameters: 
conv_featmap=[32, 64, 128]
fc_units=[256]
conv_kernel_size=[5, 5, 5]
pooling_size=[2, 2, 2]
l2_norm=0.01
seed=235
learning_rate=0.01
number of batches for training: 24
epoch 1 
epoch 2 
epoch 3 
epoch 4 
epoch 5 
Best validation accuracy! iteration:100 accuracy: 43.4%
epoch 6 
epoch 7 
epoch 8 
epoch 9 
Best validation accuracy! iteration:200 accuracy: 65.9%
epoch 10 
epoch 11 
epoch 12 
epoch 13 
Best validation accuracy! iteration:300 accuracy: 69.6%
epoch 14 
epoch 15 
epoch 16 
epoch 17 
Best validation accuracy! iteration:400 accuracy: 73.9%
epoch 18 
epoch 19 
epoch 20 
epoch 21 
Best validation accuracy! iteration:500 accuracy: 75.2%
epoch 22 
epoch 23 
epoch 24 
epoch 25 
Best validation accuracy! iteration:600 accuracy: 76.0%
epoch 26 
epoch 27 
epoch 28 
epoch 29 
epoch 30 
Best validation accuracy! iteration:700 accuracy: 77.1%
epoch 31 
epoch 32 
epoch 33 
epoch 34 
epoch 35 
epoch 36 
epoch 37 
epoch 38 
epoch 39 
epoch 40 
epoch 41 
epoch 4

## Save the best model

In [45]:
path = os.listdir('data/test')

In [46]:
X_test = []
for path_test in path:
    arr = imageio.imread(path_test) 
    X_test.append(arr[::4,::4])

In [57]:
from neuralnets.CNN import predict
tf.reset_default_graph()
result = predict(X_test,"lenet_1509803802.meta")

INFO:tensorflow:Restoring parameters from model/lenet_1509837508
tf_input: inputs/Placeholder:0
[<tf.Operation 'inputs/Placeholder' type=Placeholder>, <tf.Operation 'inputs/Placeholder_1' type=Placeholder>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/shape' type=Const>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/min' type=Const>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/max' type=Const>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/sub' type=Sub>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform/mul' type=Mul>, <tf.Operation 'conv_layer_0/conv_kernel_0/Initializer/random_uniform' type=Add>, <tf.Operation 'conv_layer_0/conv_kernel_0' type=VariableV2>, <tf.Operation 'conv_layer_0/conv_kernel_0/Assign' type=Assign>, <tf.Operation 'conv_layer_0/conv_kernel_0/read' t

## Generate .csv file

In [61]:
import csv
with open('predicted.csv','w',newline='') as csvfile:
    fieldnames = ['Id','label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()    
    for index,l in enumerate(result):
        filename = str(index)
        label = str(l)
        writer.writerow({'Id': filename, 'label': label})